In [8]:
import pandas as pd
from pathlib import Path

import re
from datetime import datetime
import locale




In [14]:
# Chemin vers le fichier avis.csv
avis_path = Path("../../data/avis.csv")

# Charger les données
if avis_path.exists():
    avis_data = pd.read_csv(avis_path)
    print(f"Fichier chargé avec {len(avis_data)} lignes.")
else:
    raise FileNotFoundError(f"Le fichier {avis_path} n'existe pas.")

# Réinitialiser les ID des avis
avis_data["id_avis"] = range(1, len(avis_data) + 1)

# Afficher les premières lignes pour vérifier
print(avis_data.head())

# Sauvegarder les données prétraitées dans le même fichier ou un nouveau fichier
avis_data.to_csv(avis_path, index=False, encoding="utf-8")
print(f"Les données prétraitées ont été enregistrées dans {avis_path}.")


Fichier chargé avec 1580 lignes.
   id_avis  id_restaurant       nom_utilisateur  note_restaurant  \
0        1              1  Companion43659771362              4.0   
1        2              1               klinkou              5.0   
2        3              1                 JoCHD              5.0   
3        4              1             Tiffany M              5.0   
4        5              1           stéphanie c              1.0   

                   date_avis                         titre_avis  \
0  Rédigé le 6 décembre 2024  "Kuma" ma petite cantine du 6 éme   
1   Rédigé le 3 octobre 2024                       Top, allez-y   
2    Rédigé le 10 avril 2024                 UNE BONNE SURPRISE   
3   Rédigé le 3 février 2024                       Super moment   
4      Rédigé le 5 août 2023  Horrible malgré un cadre agréable   

                                        contenu_avis  
0  Agréable petit restaurant à la déco typique,la...  
1  Ce restaurant est une magnifique découvert

In [16]:
# Assurez-vous de configurer la langue locale en français pour reconnaître les mois en français
#locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Pour Linux/Mac
locale.setlocale(locale.LC_TIME, 'French_France.1252')  # Pour Windows, si nécessaire

def preprocess_avis(data: pd.DataFrame) -> pd.DataFrame:
    """
    Nettoie et restructure les données des avis.
    
    Args:
        data (pd.DataFrame): Le DataFrame contenant les avis.
    
    Returns:
        pd.DataFrame: Le DataFrame nettoyé et restructuré.
    """
    # Nettoyage des noms de colonnes
    data.columns = [col.strip() for col in data.columns]
    
    # Suppression des espaces et ponctuations bizarres dans les champs texte
    def clean_text(text):
        if isinstance(text, str):
            text = re.sub(r'\s+', ' ', text)  # Réduit plusieurs espaces à un seul
            text = re.sub(r'[\"\'”“‘’]', '', text)  # Supprime les guillemets bizarres
            text = text.strip()  # Supprime les espaces en début et fin
        return text
    
    # Appliquer clean_text aux colonnes texte
    for col in ['nom_utilisateur', 'titre_avis', 'contenu_avis']:
        if col in data.columns:
            data[col] = data[col].apply(clean_text)
    
    # Conversion de la date_avis au format datetime
    if 'date_avis' in data.columns:
        def parse_date(date):
            try:
                # Supprimer "Rédigé le " et convertir la date
                cleaned_date = re.sub(r'^Rédigé le ', '', date).strip()
                return datetime.strptime(cleaned_date, '%d %B %Y')  # Format français : 6 décembre 2024
            except ValueError:
                return pd.NaT  # Retourne NaT si le parsing échoue
        
        data['date_avis'] = data['date_avis'].apply(parse_date)
    
    # Réinitialisation des IDs d'avis
    if 'id_avis' in data.columns:
        data['id_avis'] = range(1, len(data) + 1)
    
    return data


In [17]:
# Appliquer le prétraitement
avis_cleaned = preprocess_avis(avis_data)

# Vérification rapide des résultats
print(avis_cleaned.head())



   id_avis  id_restaurant       nom_utilisateur  note_restaurant  date_avis  \
0        1              1  Companion43659771362              4.0 2024-12-06   
1        2              1               klinkou              5.0 2024-10-03   
2        3              1                 JoCHD              5.0 2024-04-10   
3        4              1             Tiffany M              5.0 2024-02-03   
4        5              1           stéphanie c              1.0 2023-08-05   

                          titre_avis  \
0    Kuma ma petite cantine du 6 éme   
1                       Top, allez-y   
2                 UNE BONNE SURPRISE   
3                       Super moment   
4  Horrible malgré un cadre agréable   

                                        contenu_avis  
0  Agréable petit restaurant à la déco typique,la...  
1  Ce restaurant est une magnifique découverte. L...  
2  Lendroit ne paie pas de mine mais les plats so...  
3  Un très bon moment, les plats sont très bons e...  
4  1ère e

In [18]:
# Définir le chemin pour le nouveau fichier
avis_path_cleaned = Path("../../data/avis_cleaned.csv")

# Sauvegarder les données nettoyées dans un nouveau fichier
avis_cleaned.to_csv(avis_path_cleaned, index=False, encoding="utf-8")
print(f"Données nettoyées sauvegardées dans {avis_path_cleaned}.")


Données nettoyées sauvegardées dans ..\..\data\avis_cleaned.csv.
